In [1]:
import requests
import yaml
import re

Nomenclature maintaince has to happen somewhere  
Either early or late.  
Although Ensembl is very lilely to be current     
Monarch tends to standardise on NCBIGene IDs for other things 

[HGNC API doc](https://www.genenames.org/help/rest-web-service-help)  
and  
[Bulk Download](https://www.genenames.org/cgi-bin/download)

in this process, the time to convert ids and symbols is after
the set of tissue is associated with a gene  
b/w then it is once per profile, instead of per measurement.

In [2]:
# https://www.genenames.org/cgi-bin/download?
# col=gd_app_sym
# col=gd_pub_eg_id
# col=gd_pub_ensembl_id
# col=md_eg_id&
# col=md_ensembl_id
# status=Approved
# status_opt=2
# order_by=gd_pub_eg_id
# format=text
# submit=submit

HGNCURL='http://www.genenames.org/cgi-bin/download?'

query = "&".join([
    'col=gd_pub_ensembl_id',
    'col=md_ensembl_id',
    'col=gd_pub_eg_id',
    'col=md_eg_id',
    'col=gd_app_sym',
    'order_by=gd_pub_ensembl_id',
    'status=Approved',
    'status_opt=2',
    'format=text',
    'submit=submit'
])

In [3]:
response = requests.get(HGNCURL + query)

In [4]:
if response.status_code == requests.codes.ok:
    hgnc = response.text.splitlines()
    hgnc.pop(0)  # header
else: 
    print('ERROR ' + response.url + ' returned '+ str(response.status_code))

In [5]:
ensg_sym = {}  # in case there are any not mapped to ncbi as well
sym_ensg = {}
ensg_ncbi = {}
ncbi_sym = {}

sym_ncbi = {}

for line in hgnc:
    (ens1, ens2, eg1, eg2, symbol) = re.split(r'\t',line)
    ensg = ens2 if ens1 == '' else ens1
    ncbi = eg2 if eg1 == '' else eg1    
    if ensg != '': 
        if symbol != '':
            ensg_sym[ensg] = symbol
            sym_ensg[symbol] = ensg # to search network for our FA genes
        if ncbi != '':
            ensg_ncbi[ensg] = ncbi
    if ncbi != '':
        ncbi_sym[ncbi] = symbol
        sym_ncbi[symbol] = ncbi

In [9]:
print('ensg_sym', len(ensg_sym))
print('ncbi_sym', len(ncbi_sym)) 
print('ensg_ncbi', len(ensg_ncbi))

ensg_sym 36868
ncbi_sym 41155
ensg_ncbi 36860


There are tens of thousands BGEE measurements which won't map one to one to HGNC symbols.  

converting early would 
    reduce 
  

note:  
we can (should) never get an empty gene symbol from the nomenclature athority

as there are slightly more ensembl->symbol than ensembl->ncbi  
 - first try ensembl->symbol   
 - then ensembl->ncbi->symbol 

In [8]:
with open('ensgid_symbol_hgnc.yaml', 'w') as fh:
    yaml.dump(ensg_sym, fh)
with open('ncbi_symbol_hgnc.yaml', 'w') as fh:
    yaml.dump(ncbi_sym, fh)    
with open('ensg_ncbi.yaml', 'w') as fh:
    yaml.dump(ensg_ncbi, fh)